<a href="https://colab.research.google.com/github/ruany-doehnert/Deforestation_Amazon/blob/main/deforestation_polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install geemap geopandas earthengine-api shapely scikit-learn pandas matplotlib seaborn


In [ ]:
from zipfile import ZipFile
import os

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Path where your files are stored
folder_path = '/content/drive/MyDrive/deforestaion_amazon'

# Create zip
zip_path = os.path.join(folder_path, 'yearly_deforestation.zip')
with ZipFile(zip_path, 'w') as zipf:
    for ext in ['.shp', '.shx', '.dbf', '.prj']:
        zipf.write(os.path.join(folder_path, 'yearly_deforestation' + ext),
                   arcname='yearly_deforestation' + ext)

print("ZIP created at:", zip_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ZIP created at: /content/drive/MyDrive/deforestaion_amazon/yearly_deforestation.zip


In [3]:
# conect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import ee
ee.Authenticate()
ee.Initialize(project='amazon-deforestation-462101')

# Load labeled deforestation samples
labeled_def = ee.FeatureCollection('projects/amazon-deforestation-462101/assets/yearly_deforestation')

# Define region of interest: selected Brazilian Amazon states
brazil_states = ['Acre', 'Amazonas', 'Pará', 'Rondônia', 'Roraima']
region = ee.FeatureCollection("FAO/GAUL/2015/level1") \
    .filter(ee.Filter.inList('ADM1_NAME', brazil_states)) \
    .geometry()

# Load and process Landsat 8 imagery
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2021-01-01', '2021-12-31') \
    .filterBounds(region) \
    .map(lambda img: img.multiply(0.0000275).add(-0.2)) \
    .median()

# Calculate NDVI and NBR indices
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
nbr = landsat.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')

# Select useful bands only
selected_features = landsat.select(['SR_B3', 'SR_B4', 'SR_B5']) \
    .addBands([ndvi, nbr, ee.Image('USGS/SRTMGL1_003').rename('elevation')])

In [5]:
# Use buffered geometry to reduce complexity of non-deforestation region
buffered_def_geom = labeled_def.geometry().buffer(1000)  # 1km buffer to avoid edge overlap
non_def_area = region.difference(buffered_def_geom, 1)

# Sample non-deforestation points (negative class)
non_def_samples = selected_features.sample(
    region=non_def_area,
    scale=30,
    numPixels=2000,  # smaller number for faster export
    seed=42,
    geometries=False  # remove geometries to reduce size
).map(lambda f: f.set('Class', 0))

# Prepare labeled deforestation points (positive class)
deforestation_samples = labeled_def.map(lambda f: f.set('Class', 1).setGeometry(None))

# Merge both datasets
training_samples = deforestation_samples.merge(non_def_samples)


In [6]:
# Export training data to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=training_samples,
    description='export_training_dataset_v2',
    folder='earthengine_export',
    fileNamePrefix='training_data_amazon_v2',
    fileFormat='CSV'
)

task.start()
print("🚀 Export started! Check your Google Drive shortly.")


🚀 Export started! Check your Google Drive shortly.
